In [ ]:
#@title Install Dependencies
import importlib.util # Import the importlib module

if importlib.util.find_spec('tabulate') is None:
  %pip install tabulate

if importlib.util.find_spec('timm') is None:
  %pip install timm

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import numpy as np
import os
import yaml
import time
import re

import torch
from transformers import AutoTokenizer, AutoModel
import math
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

from pathlib import Path